
## MSM of Brownian dynamics simulations of diffusion on a 2D surface
Here we analyze simulations on another simple mode system, but one that goes beyond one dimension. Specifically, we use the model by [Berezhkovskii et al, *JCP* (2014)](http://dx.doi.org/10.1063/1.4902243). We run brownian dynamics simulations on this surface and build a simple Markov state model from it. The data can be downloaded from [OSF](https://osf.io/a2vc7/).

As always we start by importing some relevant libraries.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import h5py
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.25)
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})

#### Discretization

Here we upload the data obtained from Brownian Dynamics simulations of isotropic diffusion on a 2D potential.

In [ ]:
h5file = "../datafiles/brownian_dynamics/cossio_kl1.3_Dx1_Dq1.h5"
f = h5py.File(h5file, 'r')
data = np.array(f['data'])
f.close()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(10,3), sharex=True,sharey=False)
ax[0].plot(data[:,0],data[:,1],'.', markersize=1)
ax[1].plot(data[:,0],data[:,2],'g.', markersize=1)
ax[0].set_ylim(-10,10)
ax[1].set_xlim(0,25000)
ax[0].set_ylabel('x')
ax[1].set_ylabel('y')
ax[1].set_xlabel('Time')
plt.tight_layout(h_pad=0)

Clearly the system interconverts between two states. Both coordinates, x and y, are highly correlated, although the free energy landscape, which we can estimate from a Boltzmann inversion, varies a bit depending on the projection we use.

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
hist, bin_edges = np.histogram(data[:,1], bins=np.linspace(-9,9,25), \
                               density=True)
bin_centers = [0.5*(bin_edges[i]+bin_edges[i+1]) \
               for i in range(len(bin_edges)-1)]
ax.plot(bin_centers, -np.log(hist), lw=3, label="x")
hist, bin_edges = np.histogram(data[:,2], bins=np.linspace(-9,9,25), \
                               density=True)
bin_centers = [0.5*(bin_edges[i]+bin_edges[i+1]) \
               for i in range(len(bin_edges)-1)]
ax.plot(bin_centers, -np.log(hist), lw=3, label="y")
ax.set_xlim(-7,7)
ax.set_ylim(1,9)
ax.set_xlabel('coordinate')
ax.set_ylabel('PMF ($k_BT$)')
ax.legend()

We can also represent the energy landscape in two dimensions:

In [ ]:
H, x_edges, y_edges = np.histogram2d(data[:,1],data[:,2], \
            bins=[np.linspace(-9,9,25), np.linspace(-9,9,25)])

fig, ax = plt.subplots(figsize=(5,4.5))
pmf = -np.log(H.transpose())
pmf -= np.min(pmf)
cs = ax.contourf(pmf, extent=[x_edges.min(), x_edges.max(), \
                     y_edges.min(), y_edges.max()], \
                   levels=np.arange(0, 6.5,0.5), alpha=0.75)
cbar = plt.colorbar(cs)
ax.set_xlim(-7,7)
ax.set_ylim(-7,7)
ax.set_yticks(range(-5,6,5))
ax.set_xlabel('$x$', fontsize=18)
ax.set_ylabel('$y$', fontsize=18)
plt.tight_layout()

To construct the MSM, we assigning frames to microstates. We first need to import the function that makes the grid.

In [ ]:
from scipy.stats import binned_statistic_2d

In [ ]:
statistic, x_edge, y_edge, binnumber = \
    binned_statistic_2d(data[:,1],data[:,2],None,'count', \
                        bins=[np.linspace(-9,9,25), np.linspace(-9,9,25)])

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))

grid = ax.imshow(-np.log(statistic.transpose()),origin="lower",cmap=plt.cm.rainbow)

cbar = plt.colorbar(grid)
ax.set_yticks(range(0,20,5))
ax.set_xticks(range(0,20,5))
ax.set_xlabel('$x_{bin}$', fontsize=20)
ax.set_ylabel('$y_{bin}$', fontsize=20)
plt.tight_layout()

In this way, the continuous coordinates x and y are mapped onto a discrete microstate space.

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(10,6),sharex=True)
plt.subplots_adjust(wspace=0, hspace=0)
ax[0].plot(range(0,len(data[:,1])),data[:,1])
ax[1].plot(range(0,len(data[:,2])),data[:,2],color="g")
ax[2].plot(binnumber)
ax[0].set_ylabel('x')
ax[1].set_ylabel('y')
ax[2].set_ylabel("s")
ax[2].set_xlabel("time (ps)")
ax[2].set_xlim(0, 1500)

In [ ]:
from mastermsm.trajectory import traj

We then pass the discrete trajectory to the ``traj`` module to generate an instance of the ``TimeSeries`` class. Using some of its methods, we are able to generate and sort the names of the microstates in the trajectory, which will be useful later.

In [ ]:
distraj = traj.TimeSeries(distraj=list(binnumber), dt=1)
distraj.find_keys()
distraj.keys.sort()

### Master Equation Model 
After generating the discrete trajectory, we can build the master equation model, for which we use the ``msm`` module.

In [ ]:
from mastermsm.msm import msm

First of all, we will create an instance of the SuperMSM class, which will be useful to produce and validate dynamical models. We pass two arguments: the "discrete trajectory" that we have generated above and a value for the boolean sym. This only tells the program that it can symmetrize the data, as we are assuming our trajectory is long enough as to consider it equilibrium sampling.

In [ ]:
msm_2D = msm.SuperMSM([distraj], sym=True)

We then check the dependence of the slowest relaxation times of the system, $\tau$ with respect to the choice of lag time $\Delta t$. These can be accessed as the `tauT` corresponding to the `MSM` instance. We find that they are very well converged even from the shortest value of $\Delta t$.

In [ ]:
for i in [1, 2, 5, 10, 20, 50, 100]:
    msm_2D.do_msm(i)
    msm_2D.msms[i].do_trans(evecs=True)
    msm_2D.msms[i].boots()

In [ ]:
tau_vs_lagt = np.array([[x,msm_2D.msms[x].tauT[0], \
                         msm_2D.msms[x].tau_std[0]] \
               for x in sorted(msm_2D.msms.keys())])

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(tau_vs_lagt[:,0],tau_vs_lagt[:,1],fmt='o-', \
            yerr=tau_vs_lagt[:,2], markersize=10)
ax.fill_between(tau_vs_lagt[:,0],tau_vs_lagt[:,1]+tau_vs_lagt[:,2], \
                tau_vs_lagt[:,1]-tau_vs_lagt[:,2], alpha=0.1)
ax.set_xlabel(r'$\Delta$t', fontsize=16)
ax.set_ylabel(r'$\tau$', fontsize=16)
ax.set_xlim(0.8,120)
ax.set_ylim(50,1000)
ax.set_yscale('log')
ax.set_xscale('log')
plt.tight_layout()

Clearly, there is no dependence of the relaxation times $\tau$ on the lag time $\Delta$t.


#### Estimation

In [ ]:
lt=2
plt.figure()
plt.imshow(msm_2D.msms[lt].trans, interpolation='none', \
    origin="lower")
plt.ylabel('$\it{i}$')
plt.xlabel('$\it{j}$')
plt.colorbar()
plt.figure()
plt.imshow(np.log(msm_2D.msms[lt].trans), interpolation='none', \
    origin="lower")
plt.ylabel('$\it{i}$')
plt.xlabel('$\it{j}$')
plt.colorbar()

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(range(1,12),msm_2D.msms[lt].tauT[0:11], fmt='o-', \
            yerr= msm_2D.msms[lt].tau_std[0:11], ms=10)
ax.set_xlabel('Eigenvalue')
ax.set_ylabel(r'$\tau_i$ [ns]') 

The first mode captured by $\lambda_1$ is significantly slower than the others. That mode, which is described by the right eigenvector $\psi^R_1$ as the transition of the protein between the folded and unfolded states.

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(msm_2D.msms[2].rvecsT[:,1])
ax.fill_between(range(len(msm_2D.msms[lt].rvecsT[:,1])), 0, \
                msm_2D.msms[lt].rvecsT[:,1], \
                where=msm_2D.msms[lt].rvecsT[:,1]>0,\
                facecolor='c', interpolate=True,alpha=.4)
ax.fill_between(range(len(msm_2D.msms[lt].rvecsT[:,1])), 0, \
                msm_2D.msms[lt].rvecsT[:,1], \
                where=msm_2D.msms[lt].rvecsT[:,1]<0,\
                facecolor='g', interpolate=True,alpha=.4)
ax.set_ylabel("$\Psi^R_1$")
plt.show()

The projection of $\psi^R_1$ on the 2D grid shows the transitions between the two conformational states (red and blue).

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,5),sharey=True,sharex=True)
rv_mat = np.zeros((25,25), float)
for i in [x for x in zip(msm_2D.msms[lt].keep_keys, \
                         msm_2D.msms[lt].rvecsT[:,1])]:
    unr_ind=np.unravel_index(i[0],(26,26))    
    rv_mat[unr_ind[0]-1,unr_ind[1]-1] = -i[1]
ax[0].imshow(rv_mat.transpose(), interpolation="none", \
             cmap='bwr',origin="lower")
ax[1].imshow(-np.log(statistic.transpose()), \
             cmap=plt.cm.rainbow,origin="lower")
ax[1].set_yticks(range(0,26,5))
ax[1].set_xticks(range(0,26,5))
plt.tight_layout()